In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv("data/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [12]:
df.drop(labels=["id"], axis=1,inplace=True)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [13]:
# Independent and Dependent Features
X = df.drop(labels=["price"],axis=1)
Y = df.price

In [14]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [15]:
Y.head()

0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64

In [16]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_col=X.select_dtypes(include="object").columns
numerical_col=X.select_dtypes(exclude="object").columns

In [23]:
# Define the Custom ranking for each Ordinal Variable
cut_categories=["Fair", "Good", "Very Good", "Premium", "Ideal"]
color_categories=["D","E","F","G","H","I","J"]
clarity_categories=["I1","SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]

In [18]:
#pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [20]:
from sklearn.impute import SimpleImputer #Handling Missing Values
from sklearn.preprocessing import StandardScaler #Feature Scaling
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [31]:
# Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)


# Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinalencoder", OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
        ("scaler", StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
        ("num_pipeline", num_pipeline, numerical_col),
        ("cat_pipeline", cat_pipeline, categorical_col)
    ])

In [32]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [33]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,Y,test_size=0.30,random_state=42)

In [34]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(135501, 9)
(58072, 9)
(135501,)
(58072,)


In [37]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [39]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127


In [40]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.629077,0.258230,-0.120630,-0.600482,-0.581521,-0.572248,0.874100,-1.552614,-0.648127
1,2.605374,-2.148014,-0.120630,2.126042,2.198832,1.959219,-1.137644,0.294987,-1.314417
2,-1.125026,-1.222536,0.921902,-1.374347,-1.414721,-1.469110,-0.131772,-0.936747,2.017037
3,-1.017211,-0.574701,0.921902,-1.158385,-1.161138,-1.194265,-0.131772,1.526720,2.017037
4,0.858771,0.628421,-0.641897,0.947248,0.985258,1.004495,0.874100,0.910853,-0.648127


In [42]:
# Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [43]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [44]:
reg.coef_

array([ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
        -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
         652.10059539])

In [45]:
reg.intercept_

3976.878738902296

In [47]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true,predicted)
    return mae,rmse,r2

In [62]:
# Train Multiple Model
models = {
    "linearRegression":LinearRegression(),
    "lasso" : Lasso(),
    "ridge" : Ridge(),
    "elasticnet" : ElasticNet()
}

model_list=[]
r2_list = []

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    
    #Makeing prediction
    y_pred = model.predict(X_test)
    mae, rmse, r2 = evaluate_model(y_test,y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print("Model Training Performance")
    print(f"RMSE : {rmse}")
    print(f"MAE : {mae}")
    print(f"R2_Score : {r2}")
    r2_list.append(r2)
    print("="*40)

linearRegression
Model Training Performance
RMSE : 1029473.3531156846
MAE : 675.0758270067483
R2_Score : 0.9362906819996049
lasso
Model Training Performance
RMSE : 1029533.1506505491
MAE : 676.2421173665509
R2_Score : 0.9362869814082755
ridge
Model Training Performance
RMSE : 1029482.8101268951
MAE : 675.1077629781341
R2_Score : 0.9362900967491632
elasticnet
Model Training Performance
RMSE : 2351174.871397875
MAE : 1060.9432977143008
R2_Score : 0.8544967219374031


In [63]:
print(model_list)

['linearRegression', 'lasso', 'ridge', 'elasticnet']


In [64]:
print(r2_list)

[0.9362906819996049, 0.9362869814082755, 0.9362900967491632, 0.8544967219374031]
